[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lsizaguirre/09MIAR-TFM/blob/main/notebooks/BERT-question-answering/BERT-question-answering-fine-tunning_multi.ipynb)

<div>
<img src="https://www.dropbox.com/s/crnfuz6kotkjzxa/viu_logo.png?dl=1" width="300"/>
</div>

## 09MIAR - TFM
### Question Answering

*por Luis Arturo Izaguirre Viera - lsizaguirre@gmail.com*

---

# Librerías y Entorno


## Instalando librerías

In [ ]:
!pip install sentencepiece
!pip install tf-models-official
!pip install tf-nightly

  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Attempting uninstall: tf-estimator-nightly
    Found existing installation: tf-estimator-nightly 2.10.0.dev2022050508
    Uninstalling tf-estimator-nightly-2.10.0.dev2022050508:
      Successfully uninstalled tf-estimator-nightly-2.10.0.dev2022050508
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220427 requires tf-estimator-nightly~=2.10.0.dev, but you have tf-estimator-nightly 2.8.0.dev2021122109 which is incompatible.


  Using cached tf_estimator_nightly-2.10.0.dev2022050508-py2.py3-none-any.whl (438 kB)
  Attempting uninstall: tf-estimator-nightly
    Found existing installation: tf-estimator-nightly 2.8.0.dev2021122109
    Uninstalling tf-estimator-nightly-2.8.0.dev2021122109:
      Successfully uninstalled tf-estimator-nightly-2.8.0.dev2021122109
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, but you have tf-estimator-nightly 2.10.0.dev2022050508 which is incompatible.


## Importando líbrerias

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file

from official.nlp import optimization

from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions

In [ ]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

## Obteniendo información del entorno

In [ ]:
tf.__version__

'2.10.0-dev20220427'

In [ ]:
! python --version

Python 3.7.13


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF Version:  2.10.0-dev20220427
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


# Datos

## Carga de Dataset

In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/train-v1.1.json",
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/vocab.txt",
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/train-v1.1.tf_record")

In [ ]:
with tf.io.gfile.GFile("/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [ ]:
# Valor recomendado por Google dado lo complicado que podria ser procesar si la entrada es muy grande 
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=True)

# Modelado, Fine-Tunning

## Capa Squad

Google recomienda que a la hora de crear una capa densa al final para hacer fine-tunning nos recomienda una normal truncada con una desviacion estandar pequeña.

In [ ]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] 
    return unstacked_logits[0], unstacked_logits[1]

## Modelo completo

In [ ]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    def apply_bert(self, inputs):
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

# Fase 4: Entrenamiento

## Creación de la IA

In [ ]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 3000 # entrenando solo con 3000 lotes
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [ ]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [ ]:
bert_squad = BERTSquad()

In [ ]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

In [ ]:
def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True)
    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)
    
    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

    return total_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

In [ ]:
next(iter(train_dataset_light))

({'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[ 101, 2726, 2358, ...,    0,    0,    0],
         [ 101, 2054, 7017, ...,    0,    0,    0],
         [ 101, 2054, 2711, ...,    0,    0,    0],
         [ 101, 2129, 2116, ...,    0,    0,    0]], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 20, 122,  29,  28], dtype=int32)>,
  'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 18, 119,  20,  28], dtype=int32)>})

In [ ]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [ ]:
bert_squad.summary()

Model: "bert_squad"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    multiple                  109482241 
                                                                 
 bert_squad_layer (BertSquad  multiple                 1538      
 Layer)                                                          
                                                                 
Total params: 109,483,779
Trainable params: 109,483,778
Non-trainable params: 1
_________________________________________________________________


In [ ]:
checkpoint_path = "./content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/ckpt_bert_squad/"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

## Entrenamiento personalizado

In [ ]:
for epoch in range(NB_EPOCHS):
    print("Inicio del Epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    
    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape:
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)
        
        gradients = tape.gradient(loss, bert_squad.trainable_variables)
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))
        
        train_loss(loss)
        
        if batch % 50 == 0:
            print("Epoch {} Lote {} Pérdida {:.4f}".format(
                epoch+1, batch, train_loss.result()))
        
        if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Guardando checkpoint para el epoch {} en el directorio {}".format(epoch+1,
                                                                ckpt_save_path))
    print("Tiempo total para entrenar 1 epoch: {} segs\n".format(time.time() - start))

Inicio del Epoch 1
Epoch 1 Lote 0 Pérdida 5.7400
Guardando checkpoint para el epoch 1 en el directorio ./content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/ckpt_bert_squad/ckpt-1
Epoch 1 Lote 50 Pérdida 5.6746
Epoch 1 Lote 100 Pérdida 5.2875
Epoch 1 Lote 150 Pérdida 4.8045
Epoch 1 Lote 200 Pérdida 4.4377
Epoch 1 Lote 250 Pérdida 4.0239
Epoch 1 Lote 300 Pérdida 3.7401
Epoch 1 Lote 350 Pérdida 3.5493
Epoch 1 Lote 400 Pérdida 3.3549
Epoch 1 Lote 450 Pérdida 3.1984
Epoch 1 Lote 500 Pérdida 3.0346
Guardando checkpoint para el epoch 1 en el directorio ./content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/ckpt_bert_squad/ckpt-2
Epoch 1 Lote 550 Pérdida 2.8972
Epoch 1 Lote 600 Pérdida 2.7942
Epoch 1 Lote 650 Pérdida 2.7068
Epoch 1 Lote 700 Pérdida 2.6264
Epoch 1 Lote 750 Pérdida 2.5467
Epoch 1 Lote 800 Pérdida 2.4746
Epoch 1 Lote 850 Pérdida 2.4225
Epoch 1 Lote 900 Pérdida 2.3655
Epoch 1 Lote 950 Pérdida 2.3103
Epoch 1 Lote 1000 Pérdida 2.2423
Guardando checkpoin

# Fase 5: Evaluación

## Preparación de la evaluación

Get the dev set in the session

In [ ]:
eval_examples = read_squad_examples(
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

In [ ]:
eval_examples[0]

qas_id: 56be4db0acb8001400a502ec, question_text: Which NFL team represented the AFC at Super Bowl 50?, doc_tokens: [Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.]

Define the function that will write the tf_record file for the dev set

In [ ]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/",
                          "eval.tf_record"),
    is_training=False)

Create a tokenizer for future information needs

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

Define the function that add the features (feature is a protocol in tensorflow) to our eval_features list

In [ ]:
def _append_feature(feature, is_padding):
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

Create the eval features and the writes the tf.record file

In [ ]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature,
    batch_size=4)

In [ ]:
eval_writer.close()

Load the ready-to-be-used dataset to our session

In [ ]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/eval.tf_record",
    384,#input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training=False)

## Llevar a cabo las prediccioness

Definir un cierto tipo de colección (como un diccionario).

In [ ]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

Devuelve cada elemento del lote de salida, uno por uno.

In [ ]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(
            unique_id=unique_ids.numpy(),
            start_logits=start_logits.numpy().tolist(),
            end_logits=end_logits.numpy().tolist())

Hacemos nuestras predicciones

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs  
    unique_ids = x.pop("unique_ids")
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

0/2709
100/2709
200/2709
300/2709
400/2709
500/2709
600/2709
700/2709
800/2709
900/2709
1000/2709
1100/2709
1200/2709
1300/2709
1400/2709
1500/2709
1600/2709
1700/2709
1800/2709
1900/2709
2000/2709
2100/2709
2200/2709
2300/2709
2400/2709
2500/2709
2600/2709
2700/2709


Escribimos nuestras predicciones en un fichero JSON que funcionará con el script de evaluación.

In [ ]:
output_prediction_file = "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/predictions.json"
output_nbest_file = "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/nbest_predictions.json"
output_null_log_odds_file = "/content/gdrive/MyDrive/VIU-MIAR/09MIAR-TFM/BERT-question-answering/dataset/null_odds.json"

write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)

## Predicción casera

### Creación del diccionario de entrada

Concatenamos la pregunta y el contexto, separados por `["SEP"]`, tras la tokenización, tal cual como lo hicimos con el conjunto de entrenamiento.

Lo importante a recordar es que queremos que nuestra respuesta empiece y termine con una palabra real. Por ejemplo, la palabra "ecologically" es tokenizada como `["ecological", "##ly"]`, y si el token de fin es `["ecological"]` queremos usar la palabra "ecologically" como palabra final (del mismo modo si el token de fin es`["##ly"]`). Por eso, empezamos dividiendo nuestro contexto en palabras, y luego pasamos a tokens, recordando qué token se corresponde con qué palabra (ver la función `tokenize_context()` para más detalle).

#### Útiles varios

In [ ]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def is_whitespace(c):
    '''
    Indica si un cadena de caracteres se corresponde con un espacio en blanco / separador o no.
    '''
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [ ]:
def whitespace_split(text):
    '''
    Toma el texto y devuelve una lista de "palabras" separadas segun los 
    espacios en blanco / separadores anteriores.
    '''
    doc_tokens = []
    prev_is_whitespace = True
    for c in text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
    return doc_tokens

In [ ]:
def tokenize_context(text_words):
    '''
    Toma una lista de palabras (devueltas por whitespace_split()) y tokeniza cada
    palabra una por una. También almacena, para cada nuevo token, la palabra original
    del parámetro text_words.
    '''
    text_tok = []
    tok_to_word_id = []
    for word_id, word in enumerate(text_words):
        word_tok = tokenizer.tokenize(word)
        text_tok += word_tok
        tok_to_word_id += [word_id]*len(word_tok)
    return text_tok, tok_to_word_id

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # Convierte 1 en 0 y viceversa
    return seg_ids

In [ ]:
def create_input_dict(question, context):
    '''
    Toma una pregunta y un contexto como cadenas y devuelva un diccionario con 
    los 3 elementos necesarios para el modelo. También devuelva context_words, 
    la correspondencia context_tok a context_word ids y la longitud de 
    question_tok que necesitaremos más adelante.
    '''
    question_tok = tokenizer.tokenize(my_question)

    context_words = whitespace_split(context)
    context_tok, context_tok_to_word_id = tokenize_context(context_words)

    input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
    input_tok += ["[PAD]"]*(384-len(input_tok)) # En este caso el modelo ha sido entrenado para entradas de una longitud máxima de 384
    input_dict = {}
    input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
    input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
    input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

    return input_dict, context_words, context_tok_to_word_id, len(question_tok)

#### Creación

In [ ]:
my_context = '''Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.'''

Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.

In [ ]:
#my_question = '''What philosophy of thought addresses wealth inequality?'''
my_question = '''What are examples of economic actors?'''
#my_question = '''In a market economy, what is inequality a reflection of?'''

In [ ]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

### Predicción

In [ ]:
start_logits, end_logits = bert_squad(my_input_dict, training=False)

In [ ]:
start_logits

<tf.Tensor: shape=(1, 384), dtype=float32, numpy=
array([[-4.8681197 , -6.8374324 , -6.7977743 , -6.9242225 , -6.3793073 ,
        -6.845343  , -6.996314  , -6.3063245 , -3.831511  , -4.928098  ,
        -6.1549954 , -5.2746267 , -6.220543  , -6.460893  , -6.995262  ,
        -6.4555097 , -6.1558867 , -5.703503  , -6.9461884 , -4.621713  ,
        -6.6177297 , -6.563241  , -6.3777204 , -5.088703  , -6.949404  ,
        -5.118546  , -6.6665134 , -6.6342807 , -2.7972443 , -6.308145  ,
        -3.6453466 , -6.528621  , -4.6473436 , -6.065271  , -5.611608  ,
        -5.5659857 , -5.7484603 , -6.7741914 , -6.8635736 , -6.6369524 ,
        -6.830852  , -5.8236003 , -7.039069  , -5.995853  , -7.1023784 ,
        -6.5995407 , -3.497672  , -4.5487876 , -6.6904054 , -3.2950149 ,
        -5.9853954 , -5.188611  , -6.0134597 , -6.390878  , -6.200931  ,
        -3.4054515 , -6.5749865 , -5.034351  , -6.2924137 , -5.98198   ,
        -6.160398  , -5.524634  , -4.8826494 , -6.5172696 , -6.886624  ,
 

### Interpretación

Eliminamos los id correspondientes a la pregunta y el token ["SEP"]:

In [ ]:
start_logits_context = start_logits.numpy()[0, question_tok_len+1:]
end_logits_context = end_logits.numpy()[0, question_tok_len+1:]

In [ ]:
start_logits_context

array([-3.831511  , -4.928098  , -6.1549954 , -5.2746267 , -6.220543  ,
       -6.460893  , -6.995262  , -6.4555097 , -6.1558867 , -5.703503  ,
       -6.9461884 , -4.621713  , -6.6177297 , -6.563241  , -6.3777204 ,
       -5.088703  , -6.949404  , -5.118546  , -6.6665134 , -6.6342807 ,
       -2.7972443 , -6.308145  , -3.6453466 , -6.528621  , -4.6473436 ,
       -6.065271  , -5.611608  , -5.5659857 , -5.7484603 , -6.7741914 ,
       -6.8635736 , -6.6369524 , -6.830852  , -5.8236003 , -7.039069  ,
       -5.995853  , -7.1023784 , -6.5995407 , -3.497672  , -4.5487876 ,
       -6.6904054 , -3.2950149 , -5.9853954 , -5.188611  , -6.0134597 ,
       -6.390878  , -6.200931  , -3.4054515 , -6.5749865 , -5.034351  ,
       -6.2924137 , -5.98198   , -6.160398  , -5.524634  , -4.8826494 ,
       -6.5172696 , -6.886624  , -6.8748717 , -3.6873968 , -3.9431689 ,
       -4.877883  , -4.5075955 ,  2.523409  , -3.6225343 ,  0.37557906,
       -4.574609  , -2.884056  , -2.2662406 , -3.481987  ,  0.64

Primero la interpretación sencilla:

In [ ]:
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]

In [ ]:
end_word_id

57

"Avanzada" - nos aseguramos que el principio de la respuesta este antes del final:

In [ ]:
pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

In [ ]:
start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)]
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]

In [ ]:
start_word_id

54

Final de la respuesta:

In [ ]:
predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What are examples of economic actors?
is:
(worker, capitalist/business owner, landlord).


In [ ]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))

#### Bueno pero no tanto

In [ ]:
my_context = '''
Founded on 6 March 1902 as Madrid Football Club, the club has traditionally worn a white home kit since inception. The honorific title real is Spanish for "royal" and was bestowed to the club by King Alfonso XIII in 1920 together with the royal crown in the emblem. The team has Santiago Bernabéu Stadium in downtown Madrid since 1947. Unlike most European sporting entities, Real Madrid's members (socios) have owned and operated the club throughout its history.

The club was estimated to be worth €3.8 billion ($4.2 billion) in 2019, and it was the second highest-earning football club in the world, with a big annual revenue. The club is one of the most widely supported teams in the world.[8] Real Madrid is one of three founding members of La Liga that have never been relegated from the top division since its inception in 1929, along with Athletic Bilbao and Barcelona. The club holds many long-standing rivalries, most notably El Clásico with Barcelona and El Derbi Madrileño with Atlético Madrid.
'''

my_question = '''what year was real madrid founded?'''

my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

start_logits, end_logits = bert_squad(my_input_dict, training=False)

pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)]
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]

predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])


from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))